In [1]:
# HOW EMBEDDINGS LOOK LIKE
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

sentences = ["This is an example sentence"]

embeddings = model.encode(sentences)
print("Sentence embeddings:")
print(embeddings)

c:\ML\Self-Projects\LU_Rag_YOUTUBE\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sentence embeddings:
[[ 2.25025956e-02 -7.82918110e-02 -2.30307281e-02 -5.10001136e-03
  -8.03404376e-02  3.91321592e-02  1.13428570e-02  3.46480380e-03
  -2.94573903e-02 -1.88930202e-02  9.47433710e-02  2.92748213e-02
   3.94860059e-02 -4.63166051e-02  2.54246071e-02 -3.22000012e-02
   6.21928759e-02  1.55592253e-02 -4.67795469e-02  5.03901504e-02
   1.46113578e-02  2.31413431e-02  1.22066699e-02  2.50695981e-02
   2.93651270e-03 -4.19822074e-02 -4.01035184e-03 -2.27843970e-02
  -7.68594071e-03 -3.31090949e-02  3.22118923e-02 -2.09992286e-02
   1.16730733e-02 -9.85074118e-02  1.77932623e-06 -2.29931697e-02
  -1.31141013e-02 -2.80222856e-02 -6.99969903e-02  2.59314403e-02
  -2.89501473e-02  8.76335651e-02 -1.20919133e-02  3.98605242e-02
  -3.31381559e-02  3.59107517e-02  3.46099176e-02  6.49783909e-02
  -3.00817620e-02  6.98188394e-02 -3.99515871e-03 -1.01595558e-03
  -3.50185037e-02 -4.36567217e-02  5.08025698e-02  4.68757749e-02
   5.39663769e-02 -4.03008685e-02  3.20138643e-03  1.36

In [1]:
from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

import os
from dotenv import load_dotenv
load_dotenv()
openai_key = os.environ.get("OPENAI_KEY")

embeddings = OpenAIEmbeddings(openai_api_key=openai_key)

C:\Users\Ashi\AppData\Local\Temp\ipykernel_39876\3705299823.py:11: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_key)


In [5]:
# FUNCTION TO LOAD PDF AND SPLIT INTO PARAGRAPHS
def split_paragraphs(rawText):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=200,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False,
    )

    return  text_splitter.split_text(rawText)

def load_pdfs(pdfs):
    text_chunks = []

    for pdf in pdfs:
        reader = PdfReader(pdf)
        for page in reader.pages:
            raw = page.extract_text()
            chunks = split_paragraphs(raw)
            text_chunks += chunks
    print(" HERE IS EXAMPLE OF WHAT WE HAVE IN TEXT CHUNKS")
    print(text_chunks[:5])  # Print first 5 chunks for inspection
    return text_chunks


In [6]:
list_of_pdfs = ["1. dietary supplements - for whom.pdf"]
text_chunks = load_pdfs(list_of_pdfs)

 HERE IS EXAMPLE OF WHAT WE HAVE IN TEXT CHUNKS
['International  Journal  of \nEnvironmental Research\nand Public Health\nReview\nDietary Supplements—For Whom? The Current State of\nKnowledge about the Health Effects of Selected\nSupplement Use', 'Supplement Use\nRegina Ewa Wierzejska\n/gid00030/gid00035/gid00032/gid00030/gid00038/gid00001/gid00033/gid00042/gid00045 /gid00001\n/gid00048/gid00043/gid00031/gid00028/gid00047/gid00032/gid00046', 'Citation: Wierzejska, R.E. Dietary\nSupplements—For Whom? The\nCurrent State of Knowledge about the\nHealth Effects of Selected Supplement\nUse. Int. J. Environ. Res. Public Health', '2021 ,18, 8897. https://doi.org/\n10.3390/ijerph18178897\nAcademic Editor: Paul B. Tchounwou\nReceived: 15 July 2021\nAccepted: 21 August 2021\nPublished: 24 August 2021', 'Publisher’s Note: MDPI stays neutral\nwith regard to jurisdictional claims in\npublished maps and institutional afﬁl-\niations.\nCopyright: © 2021 by the author.\nLicensee MDPI, Basel, Switzerland

In [9]:
store = FAISS.from_texts(text_chunks, embeddings)

In [10]:
store.save_local("./myVectorStore")

In [13]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA

# Settings contains the env: OPENAI_API_KEY
openai_key = os.environ.get("OPENAI_KEY")

# Load the saved FAISS store from the disk.
store = FAISS.load_local("myVectorStore",  OpenAIEmbeddings(openai_api_key=openai_key), allow_dangerous_deserialization=True)

# Create an instance of a ChatGPT turbo model

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0, openai_api_key=openai_key)

# Build our Langchain chain instance.
chain = RetrievalQA.from_chain_type(
   llm=llm,
   retriever=store.as_retriever()
)

# Ask the LLM a question.
result = chain({"query": "What is Nepolian pizza ?"})
print(result)


{'query': 'What is Nepolian pizza ?', 'result': 'I don\'t have information on "Nepolian pizza." It seems like there might be a typo or a misunderstanding in the question. If you meant "Neapolitan pizza," it is a traditional style of pizza from Naples, Italy, known for its simple ingredients like San Marzano tomatoes, mozzarella cheese, fresh basil, and olive oil, cooked at high temperatures in a wood-fired oven.'}
